<a href="https://colab.research.google.com/github/PRAGYAMISHRA04/Time-series-analysis/blob/main/Crypto_Research_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Importing Libraries**

In [1]:
"""! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download g-research-crypto-forecasting"""
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
! unzip "/content/drive/MyDrive/Crypto-research/train.csv (1).zip"

Archive:  /content/drive/MyDrive/Crypto-research/train.csv (1).zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
from tensorflow.keras.layers import Masking, Concatenate, Add, GRU, GlobalAveragePooling1D, GlobalAvgPool1D, Dense, Dropout, Input, Bidirectional, LSTM, Conv1D, Multiply 
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Activation
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsolutePercentageError , MeanSquaredError
from keras.utils.generic_utils import get_custom_objects
import math as m
import time
import gc
import os
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential

In [4]:
"""def act_bipolar(x):
    '''Bipolar'''
    return ((1-tf.math.exp(-x)) / (1+tf.math.exp(-x)))
def act_SiLU(x):
    '''SiLU'''
    return (x/(1+tf.math.exp(-x)))
def act_softplus(x):
    '''Softplus'''
    return tf.nn.softplus(x)

"""
def act_LRELU(x):
  '''Leaky RELU'''
  return tf.nn.leaky_relu(x)
"""
def act_swish(x):
   '''Swish'''
   return x * tf.sigmoid(x)
"""
def act_Mcubic(x):
    '''Monotonic Cubic'''
    return (tf.math.pow(x,3) + x)

def act_NMcubic(x):
    '''Non Monotonic Cubic'''
    return (x - tf.math.pow(x,3))

def act_DSU(x):
    '''Decaying Sine Unit'''
    return (tf.constant(m.pi)/2)*(tf.experimental.numpy.sinc(x-tf.constant(m.pi)) - tf.experimental.numpy.sinc(x+tf.constant(m.pi))) 

def act_GCU(x):
    '''Growing Cosine Unit'''
    return x*tf.cos(x)

def act_shiftedSinc(x):
    '''Shifted Sinc'''
    return tf.constant(m.pi)*tf.experimental.numpy.sinc(x-tf.constant(m.pi))

def act_z2cosz(x):
  ''' x2cos(x)'''
  return x*x*tf.cos(x)

def act_sinz(x):
  ''' sin(x)'''
  return tf.sin(x)


"""
def act_Mish(x):
  '''Mish'''
  return x*tf.tanh(tf.math.log(1+tf.math.exp(x)))

def act_GELU(x):
  '''GELU'''
  return tf.keras.activations.gelu(x)
"""
def act_Shifted_Quadratic(x):
  '''Shifted Quadratic'''
  return (tf.math.pow(x,2) + x)
  
#get_custom_objects().update({'bipolar': Activation(act_bipolar)})
#get_custom_objects().update({'silu': Activation(act_SiLU)})
#get_custom_objects().update({'softplus': Activation(act_softplus)})
get_custom_objects().update({'lrelu': Activation(act_LRELU)})
#get_custom_objects().update({'swish': Activation(act_swish)})
get_custom_objects().update({'mcubic': Activation(act_Mcubic)})
get_custom_objects().update({'nmcubic': Activation(act_NMcubic)})
get_custom_objects().update({'dsu': Activation(act_DSU)})
get_custom_objects().update({'gcu': act_GCU})
get_custom_objects().update({'shiftedsinc': Activation(act_shiftedSinc)})
#get_custom_objects().update({'mish': Activation(act_Mish)})
#get_custom_objects().update({'gelu': Activation(act_GELU)})
get_custom_objects().update({'shiftedquadratic': Activation(act_Shifted_Quadratic)})
get_custom_objects().update({'z2cosz': Activation(act_z2cosz)})
get_custom_objects().update({'sin': Activation(act_sinz)})
class TimeCalcCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_time_start = time.time()    
    def on_epoch_end(self, epoch, logs=None):
        times.append(time.time() - self.epoch_time_start)

#**Importing Data**

In [5]:
dataset=pd.read_csv("train.csv")
dataset.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24236806 entries, 0 to 24236805
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   timestamp  int64  
 1   Asset_ID   int64  
 2   Count      float64
 3   Open       float64
 4   High       float64
 5   Low        float64
 6   Close      float64
 7   Volume     float64
 8   VWAP       float64
 9   Target     float64
dtypes: float64(8), int64(2)
memory usage: 1.8 GB


#**Data Preprocessing**

In [7]:
df=dataset.groupby("Asset_ID")
bitcoin_data=df.get_group(1)
bitcoin_data.shape

(1956282, 10)

In [8]:
bitcoin_data=bitcoin_data.drop(['Asset_ID'],axis=1)
data=bitcoin_data['VWAP']
size=data.shape[0]
train_size=m.floor(0.8*size)
val_size=m.floor(0.1*size)
train_data= data[: train_size] 
val_data = data[train_size: train_size+val_size ]
test_data = data[train_size+val_size:]
print('train:',train_data.shape)
print('valid:',val_data.shape)
print('test:',test_data.shape)

train: (1565025,)
valid: (195628,)
test: (195629,)


In [9]:
Scaler=MinMaxScaler()
train_data=Scaler.fit_transform(np.array(train_data).reshape(-1,1))
test_data=Scaler.transform(np.array(test_data).reshape(-1,1))
val_data=Scaler.transform(np.array(val_data).reshape(-1,1))

In [10]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step)]   
        dataX.append(a)
        dataY.append(dataset[i + time_step])
    return np.array(dataX), np.array(dataY)

In [11]:
train_X,train_Y = create_dataset(train_data,100)
val_X,val_Y=create_dataset(val_data,100)
test_X,test_Y=create_dataset(np.array(test_data),100)
train_X=train_X.reshape(train_X.shape[0],train_X.shape[1] , 1)
val_X = val_X.reshape(val_X.shape[0],val_X.shape[1] , 1)

In [12]:
print('train:',train_X.shape)
print('valid:',val_X.shape)
print('test:',test_X.shape)

train: (1564924, 100, 1)
valid: (195527, 100, 1)
test: (195528, 100, 1)



#**Model : LSTM**

In [13]:
def build_model(activations,shapes,optimizers):
  K.clear_session()
  model=Sequential()
  model.add(Masking(mask_value=0.0,input_shape=(shapes[0],shapes[1])))
  model.add(LSTM(32,return_sequences=True))
  model.add(Activation(activations[0]))
  model.add(GlobalAveragePooling1D())
  model.add(Dense(256,activation='lrelu'))
  model.add(Dense(128,activation='lrelu'))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error',optimizer=optimizers,metrics=['mean_absolute_percentage_error','mean_squared_error'])
  return model

In [14]:
def train_and_test_model(activations,shapes,optimizers,i):
  test_metrics={}
  #checkpoint_path = "/content/model_{}/cp-{epoch:04d}.hd5".format(activations,epoch=0)
  #checkpoint_dir = os.path.dirname(checkpoint_path)
  #checkpoint=ModelCheckpoint(checkpoint_path.format(), monitor='val_mean_squared_error', verbose=0, save_best_only=False)
  #early=EarlyStopping(monitor='val_mean_squared_error', min_delta=0, patience=10, verbose=0, mode='auto')
  model=build_model(activations,shapes,optimizers) #,optimizer,activations
  history=model.fit(x=train_X,y=train_Y,
                    validation_data=(val_X,val_Y),
                    epochs=5,batch_size=1024,
                    callbacks=[TimeCalcCallback()]) #checkpoint,early,         
  history = history.history
  model_eval=model.evaluate(test_X,test_Y,return_dict=True)
  p_test = model.predict(test_X)
  y_test_ = test_Y
  history.update({'activation_functions':len(history['val_mean_absolute_percentage_error'])*[activations],'epochs':list(range(1,len(history['val_mean_absolute_percentage_error'])+1))})
  data_list_test.append([activations,model_eval['mean_absolute_percentage_error'],model_eval['mean_squared_error']])
  del model
  return history

In [15]:
shapes=[train_X.shape[1],train_X.shape[2]]
activation_strs=['mcubic','gcu','dsu','shiftedquadratic','nmcubic','shiftedsinc','z2cosz','sin']
data_list_test=[]
data_list=[]
i=0

In [ ]:
while i<8:
  print(" Activation Layer : ",activation_strs[i])
  activations=[activation_strs[i]]
  times = []
  history_ = train_and_test_model(activations,shapes,'adam',i)
  history_.update({'time_taken_on_epoch':times})
  data_list.append(pd.DataFrame(history_))
  gc.collect()
  i+=1

 Activation Layer :  mcubic
Epoch 1/5
1529/1529 [==============================] - 653s 424ms/step - loss: 2.2888e-04 - mean_absolute_percentage_error: 4.1267 - mean_squared_error: 2.2888e-04 - val_loss: 0.0339 - val_mean_absolute_percentage_error: 6.1147 - val_mean_squared_error: 0.0339
Epoch 2/5
1529/1529 [==============================] - 666s 436ms/step - loss: 8.3287e-06 - mean_absolute_percentage_error: 1.8853 - mean_squared_error: 8.3287e-06 - val_loss: 0.0278 - val_mean_absolute_percentage_error: 5.5008 - val_mean_squared_error: 0.0278
Epoch 3/5
1529/1529 [==============================] - 668s 437ms/step - loss: 9.6537e-06 - mean_absolute_percentage_error: 2.8349 - mean_squared_error: 9.6537e-06 - val_loss: 0.0208 - val_mean_absolute_percentage_error: 4.7314 - val_mean_squared_error: 0.0208
Epoch 4/5
1529/1529 [==============================] - 660s 432ms/step - loss: 1.0179e-05 - mean_absolute_percentage_error: 1.9199 - mean_squared_error: 1.0179e-05 - val_loss: 0.0197 - val_